# Apophis Approach 2029


This notebook contains all the required code to reproduce the results and figures in the paper:

- Vallejo, Zuluaga and Chaparro, Conditions for visual and high-resolution  bistatic radar observations of Apophis in 2029.

## Environment and preparation

This section is intended to prepare the execution environment.  If running in colab, and want to 

### Required packages

In [1]:
#Evaluate if we are in Colab
try:
    import google.colab
    COLAB = True
    python="python" # In colab
except:
    COLAB = False

import sys
python=sys.executable

# Install packages
!$python -m pip install astroquery cartopy

# Get complimentary data
if COLAB:
    !git clone http://github.com/seap-udea/Apophis2029.git
    !ln -s Apophis2029/data 
    !ln -s Apophis2029/figures

     |████████████████████████████████| 4.5 MB 6.8 MB/s 
     |████████████████████████████████| 10.8 MB 15.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 30.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 36.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 832 kB 51.8 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 3.6 MB 48.9 MB/s 
     |████████████████████████████████| 219 kB 2.2 MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516265 sha256=649018f16ce04c7193eaec4448a67e0d5ec2ae1b351b62b2c9a351f967e052f7
  Stored in directory: /root/.cache/pip/wheels/98/01/f7/bd10

## Libraries and useful routines

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
import astropy.units as u
import datetime
import dateutil.parser as dparser
import cartopy.crs as ccrs
from cartopy.feature.nightshade import Nightshade
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pylab import cm

#Parameters
mpl.rcParams['font.family'] = 'Latin Modern Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['axes.linewidth'] = 2

#Constantes
DATADIR="data/"
FIGDIR="data/figures/"

## Get the ephemeris

Please explain here.

In [3]:
AU = (1*u.au).to(u.m).value
t_ini = Time('2029-04-13 13:30:00',format='iso')
t_max = Time('2029-04-13 22:05:00',format='iso')
t_end = Time('2029-04-14 06:06:00',format='iso')
epochs = {'start':t_ini.value, 'stop':t_end.value, 'step':'5m'}
apophis = Horizons(id='99942',epochs=epochs) 
ephemeris = apophis.ephemerides()
vectors = apophis.vectors()
ECUcoords = SkyCoord(ephemeris['RA'], ephemeris['DEC']
                     ,ephemeris['delta'], frame='gcrs')
vectors['delta (Rt)'] = ephemeris['delta']*u.au.to(u.R_earth)
vectors.to_pandas().to_csv("data/ephemeris.csv")

## Get Data

In [4]:
vectors = pd.read_csv(f"{DATADIR}/vectors2.csv")
ephemeris = pd.read_csv(f"{DATADIR}/ephemeris2.csv")
ephemeris['hours'] = 24*(ephemeris['datetime_jd']-ephemeris['datetime_jd'][0])
maxlons = np.loadtxt(f"{DATADIR}/maxlons.txt")
maxlats = np.loadtxt(f"{DATADIR}/maxlats.txt")
deltas = np.loadtxt(f"{DATADIR}/deltas.txt")
radio = pd.read_excel(f"{DATADIR}/AllRadiotelescopes.xlsx").dropna(how='all')
MIN = ephemeris['delta (Rt)'].argmin()

## Basic plots

## Plot approach

In [ ]:
def draw(ax,t,i,text=False,arrow=False):
    labels = list('abcdefghi')
    ax.set_global()
    ax.stock_img()

    print(ephemeris['datetime_str'][t])
    date = dparser.parse(ephemeris['datetime_str'][t].replace("A.D. ",""))
    ax.add_feature(Nightshade(date, alpha=0.2))
    ax.set_title(
        "Stages of 99942 Apophis Trajectory \n {}".format(
        str(ephemeris['datetime_str'][t])),
        size=20)

    plt.plot(maxlons,maxlats,'r-',lw=5,transform=data_crs)
    plt.plot(maxlons[t],maxlats[t],'ro',ms=15,transform=data_crs)

    plt.plot(maxlons,maxlats,'w-',lw=2,transform=data_crs)
    plt.plot(maxlons[t],maxlats[t],'wo',ms=12,transform=data_crs)

    for r,row in radio.iterrows():
        if not np.isnan(row.Power):
            plt.plot(row.Longitude,row.Latitude,'b*',ms=15,transform=data_crs,label=f"{row.Stage} "+row.Name)
        else:
            plt.plot(row.Longitude,row.Latitude,'bo',ms=10,transform=data_crs,label=f"{row.Stage} "+row.Name)
        if text:
            dx,dy = row[['dx','dy']]
            plt.text(
                row.Longitude+dx,row.Latitude+dy-1,
                row.Stage,transform=data_crs,fontsize=15)#,rotation=30)
    plt.legend(loc=3,prop={'size': 15})

    di = 0
    rot = [0, -30, 0, 5]
    dys = [0, -2, 10, -5]
    dxs = [25,0,50,10]
    f = int(len(maxlats)/4.5)
    for i in [1,2,3,4]:
        plt.text(maxlons[i*f] + dxs[i-1],maxlats[i*f] + dys[i-1],
        f"Stage {i+di}",size=20,color='black',
        transform=data_crs, rotation=rot[i-1])

data_crs = ccrs.Geodetic()

t = ephemeris['delta (Rt)'].idxmin()
# t = int(0.93*len(ephemeris))

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
draw(ax,t,0,True)

name = f"{FIGDIR}/map"
plt.savefig(name+".pdf",bbox_inches='tight')
plt.savefig(name+".png",bbox_inches='tight')

In [22]:
!make -C Apophis2029 pull

make: Entering directory '/content/Apophis2029'
Getting the lattest changes
HEAD is now at 4a413ed Commit
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0
Unpacking objects: 100% (5/5), done.
From http://github.com/seap-udea/Apophis2029
   4a413ed..cb3e208  main       -> origin/main
Updating 4a413ed..cb3e208
Fast-forward
 ApproachingPlots.ipynb       |   2 +-
 data/AllRadiotelescopes.xlsx | Bin 0 -> 80289 bytes
 2 files changed, 1 insertion(+), 1 deletion(-)
 create mode 100644 data/AllRadiotelescopes.xlsx
make: Leaving directory '/content/Apophis2029'


In [17]:
!ls data

ephemeris.csv
